In [1]:
import os
os.chdir("../../../../")
from scripts.python.PdfParse import *
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
tonga_lsts = os.listdir(os.getcwd() + "/data/tourism/tonga/")
filepaths = []
for path in tonga_lsts:
    folder_path = os.getcwd() + "/data/tourism/tonga/"
    if "Dec" in path:
        filepaths.append(folder_path + path)
    elif "2021" and "Bulletin" in path:
        filepaths.append(folder_path + path)
    else:
        pass

In [3]:
months = pd.DataFrame()

for file in filepaths[:-1]:
    print(file)

    df = load_pdf(file, "Monthly Arrival and Departure", table_page=-5)
    latest_year, year_idx, month_idx = split_time(df, "Period")
    month = df.iloc[month_idx, 0:4]
    start_year = detect_year(df.iloc[month_idx].iloc[0])

    month = (month.dropna(how="all").reset_index()
             .drop("index", axis=1))

    print(f"The file starts from {start_year}.")

    month = separate_data(month, "Air Ship", " ").drop("Air Ship", axis=1)
    month = remove_separator(month)
    month = month.replace(r'^\s*$', 0, regex=True)

    if check_quality(month, ["Period", "Year"], "Total") == False:
        name = file.split("/")[-1].split(".")[0]
        print("  ", name, "could go wrong!")

    generate_time(month, start_year)
    months = pd.concat([months, month], axis=0)

months = (months[["Year", "Period", "Air", "Ship", "Yacht", "Total"]]
          .drop_duplicates()
          .sort_values(by="Year")
          .reset_index()
          .drop("index", axis=1))

/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/Statistical-Bulletin-on-International-Arrivals-and-Departures-2021.pdf
The file starts from 2018.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/Migration-Report-Dec-2017.pdf
The file starts from 2013.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/Migration-December-Report-2019.pdf
The file starts from 2018.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/12-December-Migration-Report-2014.pdf


Got stderr: Mar 08, 2023 12:34:54 AM org.apache.pdfbox.pdmodel.PDDocument importPage
Mar 08, 2023 12:34:54 AM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/12-December-Migration-Report-2015.pdf


Got stderr: Mar 08, 2023 12:35:01 AM org.apache.pdfbox.pdmodel.PDDocument importPage
Mar 08, 2023 12:35:01 AM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/Migration-December-Report-2020.pdf
The file starts from 2018.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/12-December-Migration-Report-2013.pdf


Got stderr: Mar 08, 2023 12:35:12 AM org.apache.pdfbox.pdmodel.PDDocument importPage
Mar 08, 2023 12:35:12 AM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2010.
/Users/czhang/Desktop/pacific-observatory/data/tourism/tonga/12-Migration-Report-Dec-2016.pdf


Got stderr: Mar 08, 2023 12:35:20 AM org.apache.pdfbox.pdmodel.PDDocument importPage
Mar 08, 2023 12:35:20 AM org.apache.pdfbox.pdmodel.PDDocument importPage



The file starts from 2011.


In [4]:
# Clean the datetime format
dates = list()
for idx in months.index:
    month, year = months["Period"][idx], str(months["Year"][idx])
    if type(month) == str:
        try:
            YM = year + month
            dates.append(datetime.strptime(YM, "%Y%B"))
        except:
            dates.append(datetime.strptime(YM, "%Y%b"))
    else:
        dates.append(month)

months["date"] = dates
months = months.sort_values(by="date")

months

,Year,Period,Air,Ship,Yacht,Total,date
1,2010,January,3158,646,4,3808,2010-01-01
2,2010,February,2379,0,5,2384,2010-02-01
3,2010,March,3134,853,5,3992,2010-03-01
4,2010,April,2818,2802,30,5650,2010-04-01
0,2010,May,3670,5303,177,9150,2010-05-01
...,...,...,...,...,...,...,...
150,2021,Sep,4,NaN,NaN,4,2021-09-01
157,2021,Oct,8,NaN,NaN,8,2021-10-01
148,2021,Nov,0,NaN,NaN,0,2021-11-01
147,2021,Dec,0,NaN,NaN,0,2021-12-01


In [7]:
colnames = months.columns[~months.columns.isin(["Period"])]
months = months.dropna(subset="date")[colnames].drop_duplicates()
months = months.reset_index().drop("index", axis=1)
months.head(5)

,Year,Air,Ship,Yacht,Total,date
0,2010,3158,646,4,3808,2010-01-01
1,2010,2379,0,5,2384,2010-02-01
2,2010,3134,853,5,3992,2010-03-01
3,2010,2818,2802,30,5650,2010-04-01
4,2010,3670,5303,177,9150,2010-05-01


In [8]:
# Check for duplicates (e.g. Dec vs December in different years)

# indexes = months[colnames].drop_duplicates().index
# months = months.iloc[indexes].reset_index().drop("index", axis=1)

#Save the file
months.to_csv(os.getcwd() + "/data/tourism/tonga/tonga_monthly_visitor.csv",
              encoding="utf-8")